# OpenAI

In [1]:
%reload_ext autoreload
%autoreload 2

import sys, os, ollama, torch, logging,datetime, httpx,re, hashlib
from ollama import Client
from mangorest.mango import webapi
from openai import OpenAI
    
logger = logging.getLogger( "geoapp" )

device = "cpu"
if (torch.cuda.is_available() ):
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"

OLLAMA_HOST= 'http://127.0.0.1:11434/v1'
OPENAI_KEY = "NO KEY"

sys.path.append(os.path.expanduser("~/.django") )
if (os.path.exists(os.path.expanduser("~/.django/my_config.py"))):
    import my_config
    try:
        from my_config import OLLAMA_HOST
        OPENAI_KEY=my_config.OPENAI_KEY
    except:
        pass


2024-12-24 15:19:09,805 app.mangorest INFO: ************* MANGO REST API 2.1****************


In [2]:
#%%writefile "../openaigpt.py"
import os, sys

sys.path.append(os.path.expanduser("~/.django") )
import my_config
from my_config import *

from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key=my_config.OPENAI_KEY
)

'''response = client.completions.create(
    model="llama3.2",
    prompt="____",
    max_tokens=256
)

print(response.choices[0].text)
'''

completion = client.chat.completions.create(
  model = "llama3.2",
  messages=[
    {"role": "system"    , "content": "You are a helpful assistant."},
    {"role": "user"      , "content": "Who won the world series in 2020?"},
    {"role": "assistant" , "content": "The LA Dodgers won in 2020."},
    {"role": "user"      , "content": "Where was it played?"}
  ]
)

print(completion.choices[0].message)

2024-12-24 15:19:11,339 httpx INFO: HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletionMessage(content='The 2020 World Series was the first neutral site (Texas) in 2017 as both teams agreed to this because of COVID-19, however 2020 was still played at Globe Life Field in Arlington, Texas', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


# OLLAMA

In [75]:
%%writefile "../olgpt.py"
import sys, os, torch, logging,datetime, httpx,re, hashlib, json, base64
import ollama
from ollama import Client
from mangorest.mango import webapi
from openai import OpenAI
    
logger = logging.getLogger( "geoapp" )

device = "cpu"
if (torch.cuda.is_available() ):
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"

OLLAMA_HOST= 'http://127.0.0.1:11434/v1'
OPENAI_KEY = "NO KEY"

sys.path.append(os.path.expanduser("~/.django") )
if (os.path.exists(os.path.expanduser("~/.django/my_config.py"))):
    import my_config
    try:
        from my_config import OLLAMA_HOST
        OPENAI_KEY=my_config.OPENAI_KEY
    except:
        pass
#------------------------------------------------------------------------------------------    
import socket
from urllib.parse import urlparse

def is_host_reachable(host, timeout=0.1):
    url = urlparse(host)
    host, port = url.hostname, (url.port or 80)
    print(f"Trying {host} {port}")
    try:
        socket.create_connection((host, port), timeout)
        return True
    except ConnectionRefusedError as e:
        print("Connection refused.")
        return False
    except OSError as e:
        print("OS error: {0}".format(e))
        return False
    
if ( not os.environ.get('http_proxy','') or not is_host_reachable(os.environ['http_proxy']) ):
    proxies = None
else:
    proxies = dict(
        http_proxy = os.environ.get('http_proxy',''),
        https_proxy= os.environ.get('https_proxy',''),
        no_proxy= os.environ.get('no_proxy','')
    )
#-----------------------------------------------------------------------------------------    
def getClient(host=OLLAMA_HOST, key=None, proxies=proxies):
    client = OpenAI( base_url = host , 
                api_key = key or my_config.OPENAI_KEY,
                http_client=httpx.Client(
                    proxies=proxies,
                    transport=httpx.HTTPTransport(local_address="0.0.0.0"),
                    verify=False
                ),
            )
    return client
#-----------------------------------------------------------------------------------------    
@webapi("/gpt/generate/")
def ollma_generate(request=None, model="llama3|mistral", prompt="", stream=True,**kwargs):
    OLLAMA = Client(host=OLLAMA_HOST)
    logger.info(f"OLLAMA_HOST => {OLLAMA_HOST}  Prepping: {datetime.datetime.now()}")
    
    model = model.split("|")[0].strip()
    response = OLLAMA.generate(model=model, prompt=prompt, stream=stream)
    # Stream response
    ret = ""
    for chunk in response:
        data = chunk["response"]
        ret += data
        
    logger.debug(ret)
    return ret
#-------------------------------------------------------------------------------------------    
def tob64(imageFile):
    with open(imageFile, "rb") as f:
        b64Image = base64.b64encode(f.read()).decode("utf-8")
    return b64Image

@webapi("/gpt/llmVision/")
def llmVision(request=None, model="llama3.2-vision|llama3.2-vision:90b", 
                file="", messages=None, prompt="", **kwargs):

    OLLAMA = Client(host=OLLAMA_HOST)
    if ( request and not file and not messages):
        for f in request.FILES.getlist('file'):
            context = f.read()
            #fileIO = io.BytesIO(content)
            file = f"/tmp/{str(f)}"
            with open(file, "wb") as f:
                f.write(context)
            #context = open(file, "rb").read()
            #context = tob64(file)      
            break;      
    elif(not messages):
        #file  = "/tmp/people.jpg"
        #q = "what is in this image"
        context = open(file, "rb").read()
        #context = tob64(file)
        messages= [ 
            dict(role="user", content=prompt, images=[context])
        ]
    else:
        if ( type(messages) == str) :
            messages = json.loads(messages)

    #print(messages)
    resp = ollama.chat( model= model, messages=messages)

    return resp.message.content

#-------------------------------------------------------------------------------------------    
@webapi("/gpt/openai/")
def openaicall(**kwargs):
    return llm(**kwargs)

@webapi("/gpt/llm/")
def llm(request=None, model="llama3.2|mistral", host="", use_openai="", prompt="", 
                key="", messages="", temp=None, max_tokens=None, top_p=None, **kwargs):

    if use_openai or model.startswith('gpt'):
         host= "https://api.openai.com/v1/"
    elif not host:
        host = OLLAMA_HOST
    client = getClient(host)
        
    tmsg = [
                {"role": "system"    , "content": "You are a helpful assistant."},
                {"role": "user"      , "content": prompt or "tell a joke?"}
    ]

    if (not messages):
        messages = tmsg

    if ( type(messages) == str) :
        messages = json.loads(messages)
    

    model    = model.split("|")[0]
    
    logger.info(f"Using host: {host}, {model} : {messages}")
    
    completion = client.chat.completions.create(model=model, messages=messages, 
                                                temperature=temp, max_tokens=max_tokens )

    ret = completion.choices[0].message
    
    return ret.content
#-------------------------------------------------------------------------------------------    
@webapi("/gpt/test/")
def test(request=None, model="llama3.2|mistral", info=None, **kwargs):
    import json
    logger.info(f"{model} : info: {info} {type(info)} {kwargs}")

    if (info):
        infoj = json.loads(info)
        print(f"INFORMATION: {infoj}")
    return "OK"

#-------------------------------------------------------------------------------------------    
@webapi("/gpt/getLLMModels/")
def getLLMModels(request=None, model="llama3.2|mistral", info=None, **kwargs):

    list = []
    models = ollama.list()
    for m in models.models:
        list.append(m.model)

    list = list + "gpt-3.5-turbo gpt-4o".split()
    list.sort()
    return list
#--------------------------------------------------------------------------------------------    
# See: for voice options: https://platform.openai.com/docs/guides/text-to-speech
# alloy is our default
@webapi("/gpt/tts/")
def openaitts(request=None, prompt="", voice="alloy", fname_prefix="aud_", force="", **kwargs):
    #client = OpenAI( api_key = my_config.OPENAI_KEY)
    client = getClient(host="https://api.openai.com/v1/")

    basedir="static/data/tmp/"
    os.makedirs(basedir, exist_ok=True)
    
    fnames = []
    prompts= []
    segs = re.split("\\s*\n\\s*#\\s*---+.*\n\\s*", prompt, flags=re.MULTILINE)
    
    for i, s in enumerate(segs):
        s = s.strip();        
        if ( not s):
            continue
        
        h = hashlib.md5(s.encode()).hexdigest()
        speech_file_path = f"{basedir}{fname_prefix}_{h}.mp3"
        if ( os.path.exists(speech_file_path) and not force):
            logger.info(f"===> {speech_file_path} exists!")
            fnames.append(speech_file_path)
            continue
        
        with client.audio.speech.with_streaming_response.create(
                model="tts-1",  speed=1, voice=voice,  input=s ) as response:
            response.stream_to_file(speech_file_path)
        
        fnames.append(speech_file_path)
        prompts.append(s)
    
    return dict(voice=voice, input=prompts, files=fnames)

#-----------------------------------------------------------------------------------------    


Overwriting ../olgpt.py


# Test

In [73]:
l = " a c b d".split()
d= sorted(l)
l, d

(['a', 'c', 'b', 'd'], ['a', 'b', 'c', 'd'])

# Experiments

## Vision example

In [ ]:
import ollama, base64

model = "llama3.2-vision"
file  = "/tmp/sat.jpeg"
#context1 = open(file, "rb").read()
context = tob64(file)

file  = "/tmp/people.jpg"
context1 = tob64(file)
q = "what is in this image"

#context = '/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxITEhUSExAVFhUXFxcVFRYVFRUYFRUWFRcWFxYVFRYYHSggGBolHRUVITEiJSkrLi4uFx8zODMsNygtLisBCgoKDg0OGhAQGi0lIB0rLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0rLS0tLi0tLS0tLS0tLf/AABEIAKgBLAMBIgACEQEDEQH/xAAbAAABBQEBAAAAAAAAAAAAAAADAAECBAUGB//EAEAQAAEEAAQDBQUGAwgBBQAAAAEAAgMRBBIhMQVBUQYTImFxFDKBkcEjUqGx0fBCYuEHFSQzNENygoNUkrLC8f/EABkBAAMBAQEAAAAAAAAAAAAAAAABAgMEBf/EACQRAQEAAgIDAAICAwEAAAAAAAABAhEDIRIxQQRhIlEyQnET/9oADAMBAAIRAxEAPwDOkaq7wr0pCqSLpxr1FdygSiPQSVqki5Dc5IlQcUbBnFCcU7ihuKEouUCpEqBSKouQyplQKSagVGkSlNkVo0i3QIYrEGELiAASTsAtng/Z+SY6Cm9T9F6FwXs9HA3NQutXHdVqT24+Tnk6jkuDdi3Op0vhb05rs8DwrDwtAjYPNy2IYwRqP1WfxZ4Y0kEDy569fklLbdMLu91ajewC1n8Q4sKpm5005fBYsuMFCyTyyg9eqI9gyjKQOtuykDSzSrwk9ntZgxAadSQavXcnkPzVbF4iY6sbXm/QfHz8kZuHAcXk2QBRvqNr5JRHny3Fk1r5n4qeghGGlga8ZjfiJJAsXRratdlBzm6hkYHO/db6+e26bGR5qLHDSudtHoFF9N09487vdBmh+88/IX+eyU+MAGjtPMD6LK4jjCzfn8VUixGYWTp0tPx+m1jjwdhr6qniH88wNcuQVLE4qhQ0J5N+pWdisaaot8rJF+YKejXJ5/I/BZ2Ln5A/khe2ZednkOiFPi73FHyQqQGYmru/gqEkx5j11VmWcHSyqkjCVFqpEM17BM937CYurTT6KDn/AAUqJxrkouKRHn+CgUjQKgVJwUaUm9Lkeq73qT3qu9yuPQJz0MuTOKG5ysjuchuKRKgSgjEqBKclRJQRiVApymSTUSmDVMNV/AYB0jg1o/onJtlnnMZuqsGGLjQFldhwHstdOkHo39Vq8D4E2OjVuPM/QLrYYQ0ba9FVvi87PmvJdY+g8DgGxgChXRS4gSSBsAbKsAUbWDxXGuL+7Fb67863UYy2p8ZJpan4kyO25rrn++a5ni/GhJ4Q2upJoeXKyq8jXFxsmhvzvy6BUjk0D9dz4dvK3fRb444zs0oMQM9VnvkLHMakrcwMMTPERbjqKvLR3y/eXO+2taXPYBR8JHTYGhyNc0bEYsyU3XIPdABacp5Gz+7RlNqbc05dIAKqx4XEDlzq1dxUwqgbPS70/RcwyJoeHDMNqANHzN9PhzWk+GiMpOvSq+Pn+qzsC6KFmtvzPks/FRG/DJQu3A+anNE40M/Oz6hUsVjMttFUBbiT+JU+j0ocQewuIO9bD4/JUX47XIwgDmQN+tI75Gv1LwfNp15aac1md2OlDqTyVnINipxW+vl+qzJcRpVahPiHtJoE0PkEJsZOp2S2uQKSW/gkXCuaLI0XSHLopMCRo80IV0TSO11UHFTVGeegTFRLkNxUmk5ygXKBSU0yJUbTkqKA9Ac9Bc5RL1AuWsd53FDJSJUCUy2clQJSJUCUbI5KiSkSmSK0k7W2k1trY4Nwp0rqA05nonIx5OSYzdD4Xwx0rqA9T0XoHBeENjbQGvMqxw/hLY2ANG3zK2sBh705DevyTuUk6eXnnly5fo2Ew/JtXzJV0Yehe5+qM+mNsAKnjOItiidNJmpvJrS5zidmtaNSSsLlvtrjhJ0r4yYMaXyGgB8ST5ddapcuRK+UyBoYD9/U9KIHPyVfi3a6OR4Ps2KLWkbQ11zuBvStG35mrNKhN2wZs3CSjkA90TDf8Lcpdoeo5Dfon569NJhPoXaJj4mOccdrq4MEYGn3RqfmRr5IGA+0hEjS17jQeG7g/wAzdxdX8Vzs7Z8ZKQMgsF5c6aMNjjb78zzdiNuutWa0XXdp8JgMDHhThZgMV3cbpC97mtkhc0UZo9ac7TK1tH5BVjyyHcZVfh8bnSNgihJlJrKR15uv0PwBOyl2gwWIwsgjka0F3iGR4N3/AMaLVk8B7XzYSSSWN+DMjw4Ey+0Py8yAWAUAMuYnTwgdScqfjL3zHEPxcL3lwcSWYgk37oDcm7uTeQHNGXJLf0Jjp1U4kjAdK4HvSGAa+FwBcw3yHhINfeWkGlht4oNBP8vnX5rI4E9+L7uTESMfC2ZscbYmPYZ8UQSGNc7/AG42kvcR0As5lU7T8XlleY4jO2Foq24XPnIPilzOI8N01oG+l9Ef+k0dxldnwbg7sTG/ESSdzA27cRZ03DRtpsSbAOlE3XL4Xh/fPeA95jBOuxcOVgaLJlx+NfEzDvlxZiafDF3EbBepGYZrc1upc42M2hVM981hySYpraBI76KE5QdLpxOZ7vkPJZXLYC4qWw4mSIEgZc7dd7rQn1zfgl7VnBObQiv6FVu08LoO7gfmMxZ3+Ic9we8OdoyHNXusF2Nszj0WTwrF6uadb1HqP3+C2wy6KtaNoJ8h+aJiH0L+SHDdGhupSeI68lYNEaF8yhzOCmR5oUrUqcVJSguf5o0yrOFrOqRJUSpKKRokpipFyGUgSZIpJE7PMolyHmTWtXcmSokqJKiSgJEqJKa0yC2dSaFFoWhw7BOkcGtGpTkZZ5zGbWOD8MdK4NA9T0XpXCeFtiZlA+P6qHAuDCFoFa81uww/vzRllr08zPK8l/RsNBmoc/p1V+aWKFuprSzzJWLxjifdtLGHxHdw/IKrw/Dki3UXOPizaj0F7n4KfHc3TnXUB4xxaHJ38spjiDqDSCS55vK1rW2XvP3aPXkvPO0HHxinB8smEpoe2NubG0xvMAtAbdaPlAF+6F7NBExoDcrdLIto0J3qxp/RZ3apkhwcxhJZI1hews8LiY/HlFdaI+Ki3fXxrj08N9ogrwmEHwEFsMzz/KAx7iLI0aytBZ6lEw2EfI4Mhic4uIY1rMGx1k/wZ5D7tm3PJAPUDVCxfaTFnT2zEEHb7eXY/wDZUOJ8andlvETEBgBBlkIOribF67p5celuvxODqP2eBj54s49omgw4/wAXKyiyMd2AG4WPwnfXcbigf3fjCMzYOLbXbIHR7eGwGtq60aNmjXTddN/Z52Xx3evlZiTGzDufG0OJcx7xuRHtlIO/82ixO0GOxmKllilkz5iGvblaIyIyS2xVUDZBUeO7qBVHCuIf+m4nuBTpnMbfJpJYfs22bP8AETz3RsLwfFlzRIMdBH4s88uNa1sUY1knf9n71AgD02NFcpxjg2HYDlma6T7rGFzb6EjQLnyynUQB51+KVlhvW+1OJGIbh24VhkhZE6ONzsTFG8NeQHOJkdZlky0TyDjvemDN2anjrvMLBGTRb32OhAsaA0JQSyMVoOZC5TBsA3ILdyWau8qul1fEZpsVhnSvLQIcndsBBOQnK/MebqLXcttlvjxTLsbCPBB93hzRRbrjXF2QHNRImPie7c8geat8EjwsMomxD8A4RjvWsgklke+cUIm1mIDGXet+6eq47EylppVBncTT2N3PicG/mllx441PbT7QcTE880ubNdNB5mrLjR/mJWTwx32o+P5Kk5pB1r4EH8QtTgkPjBI15eimZb6PXbd73yUAdT+Kg93iq9ipseButRpEur4obnaJTbqDipMB4KCUZ7kF7lJoPKESpEqBSMySSekgimUqTUgnU5k1qFpWq27NpWmJTWmtGxtK0gohFibacRaPhYS4gAalen9kuBCJudw8RWN2J4DdSvGn8IP5r0OGNPLLU04OTO53XwB13XNLH4nu20N/xRZsrfGf4b/7H9FiSSCRxcXgAXqVOM32j0w8ZO50wABcbGg3vouo4fOOlVpXMVoVyOMxjWSEQl5eSQDoDnJAb16kUrOF7RFrLLPGdG0ByOUkgbag/Jb5y2CTTuIWFx03/JD4gY7yd/lPQOaCfUGyR5BcxwTjMjxK1shDi3KHCjkcQTm16UvNOO4bFOnEToSZm2MwDj3hu2yZz8835bLizkyyuHn42OrjwuvKz23O3PYfDxMa7CiZp93KA6ZoA2cW/wCYBY3bmr7vNc52X7On2yBuMpkGa3vJ8LmtF5erSdBTgCL2XVcXllkhkzzSBzIHOzxuykOYLBcRyJGvrS43s7NJiATLKTRuSR5JIjblJ8yaNAczpzWH43LnlN2tOTGTLUe58UxWFifIwWIpXBs4b3oykRtawRhg3Iy3XkuCxHZMmeaMzEQxvLdPffoHDMToPeH9Fqs/tAY5rX9w7v4wWRlzvDlcB43gbu5ab1emwq8J4qMr2vcTI4ulzO3kzmyfUbUOgW2NrGzTOl4ThYhTYW31cMzvmVxnajBRlrnNY0OGoIAGg3BrfS1ucZ4pROq5rFY2w4najfyWk/o9/E+wvBHYuXIDTWi3u6DlXmV6lFwbCxObE4NcCMpa5/ic07gNsHXXbzXGf2SSBjJzzJaPlf6lB7U417nhhBfI0AFwa4OcdNNdgDzHqKXJ/Pl5LxzPxkjowkmMtm9tbtr2Rw7nAYaGTMd+6dmPqY5CLHo4H1Xm/GeDSwOpwsEloIvcEggg6g6f/q73GcQlOGc2VzhIIwXOaSHZhl5g9avquamZTG24uOUOtxJNi7u/In5LTg8ssP5XemPLqXUjC4Xgs77I8DfE/wD4t1KnwzFEzgk+86vIXtX4K/HiA2GYDmz8yG//AHWDC/K4O6EH5G1rbqzTOOn4i0slcPO//cA76oRk6IvEZjJI51Afw6c8vhB+QCqt0W6qPI9Vw5O9yGSlQk9BepkoZSNApiE7kkAyYpyVEpEiSmTlMkHR5k+ZCtPap1bTzJWh2pNQQjAui7McIM0gB90an9FiYOEuIA3JoL1jsxwoRRtHM6lV6m3Lz8n+sbuAgDWgAUBoFb7zkNhufohl/h0HkPVVOJS5I8oOtarOTdYTqM3jvEO8JY06D8Vnx0W1dKD3jKeZPPy6KtHM6g0D5b69F0yamoliYyHxPIsmzValUYpGtOY24kaAWKJ6nc/0XUY3DmPDOljkOYOIeDo63UAxrfQk2qowTBFEGszyVbnObWlmg1t7DrzT8tq+M7hGNdDI6QVp7zDfiGYAn8Sur4ljnYhpYSYyw2KOrhvo3N4iaWFFwTEFsp9lmvKchEbzdUQBQ8q+K0OJSSxYQYgYaV8haAxgieXB7h/EALaBrfpW5XF+V+NjyXy9V18HLqaZGMw0z5GyYZ+HmjB1hkzRTSty5XwuEtNc0hx2J5dFzGNwjcNJLGyN8cZeHNZLuWDURuPSrF+Sz3YfiL7uHFnMdfsZqNn/AIphJiI3COSCW3WGNfE8F4H3Wltu35KODj8bqnllL29xx3B8HxXCDE4ZrWS5B7tAhzRrFIBzG1+i8hxMxLHQk5XsJLHDdrh9PJXsBh8RlLoYsRA7K6+6bLHmDWl1EAC9iPisni3BMVEGvMMri+6pjifPNp+PmurHj8Z2wuTnsVjnk5X6O59D5hU8ViKGUbnf06I+OwmKebdh5NNvsn/oqfsUmbKY3g70WuuutUsbl8hui7E8TMD8xvIazgdLGo8xa77iGNYBnoltaEAGwdbBu15rhMM9oru37fdd69PJXXY2eNlNEnQDK4gDnoufk/Hxzsyb4ctxml7iOOkdMHQmJzNnMzESOaRTmva+r56NtZuOYWkh2hyim620Vzvpt8Vjzd6f9t/xa5V5JHg+KwSNiK08gtuKTHplnltZJ8Eh5EAfJ7D9Qs5XY3gwuHMG/UHLf/wHzVNoRn76Q6WU7EbEX+v4oDnKf8DB5E/M/wBEIrerM9QKk9RSMrUU4TICJUSVJRKCMEzk6iUgZJJJAbWZLMh2ntDpEBRIwghXsBAXuDRuSnJtnnlqOr7D8LzyZyNBsvSmUBQ32WHwOARRtY3fn6rTkmA06J5d3Tz/AC8rtZZMASSdG7evNc/icQXh7ifRD4ljiSGN2Jv5BUpTVBaY467Pe08PC55DG8xZOtNF+849AkM0IOQnxnK2W6aRZDtdgSHD0AVKfGPDvCasZT6bqtCXSRd34i4EkNAPhZoc5PIWD8wqsONDjPGWAhrN3AF+xIdoAOmgHr4vJYsHEnNdeY2dAdTz5hCxeEdGWkjTQguFO130vzG/ULtOBRYeVhqr0dsLBrkem3yStmMVpwfagzOeyduPEXeMHgfJiGjvsPlbJQjYQAWFjiL3eVktbLsOLx1q2+/xXuyeKIn7PkV33avAQnDSjuu8dC72hjMxZmyA94Mw19wuNc8gXmntWD904J9UGf6lw8L/ABxk3Hprv0XLn/k0w9L3+Iq/73jzUHC8RidHx6Pu2cx4itPCOl9ne5uLjnmwk8eNw4bNI5/cPLY5mFzwCGf5JOv3vNcvhMZh2i3wSSOzGW+/bHqDUkZb3Z5anqANFf4JxjBQTscYJe7GaGX/ABIdmw+Ia5rrb3IzU17jVjUBSpt4nD4mJ5YeL5Qxxac+Kna8Q4hmaF7wRWdodmrrQtBlbji0j+92B2TKf8c8VLh/e5jTu9/M/FLi2BhjAZiYZXPY92BkeyURtLCTJhZgHxvtpLSc2bQMGhWYcZgaD+6xYdpMPtYTUsRyyAjuhqR4ztdDZO+yXpHY2y7+9WkZhJX94V9nJQeNX8jQHqtWFs5wsL5MY102Hnlwkk0eJEjxDjWERvc9pJblka6s1LkInYENDHHEGvsy5vdBpZL42aOF+E677hafAZsNJ32FjOIzYuAxgSd2GifD1JCSWnMHXGB/5ClsKpk4mLacY8OpzCPb49HsNkf52+lfFO7E8V1d7ZJXhk/1zKrZw/zepAryWbxQwOeJi+QGVrJyGxsc0PJyygW8H3w/psqk8UAbUcshcCWAPia1pDtW24SGuZ25DZFtVa15JeJg0cXLQcQT7Y06OGn+55FC46yaTBRSTOL5IZnxOJeJHFsgD2EuBOgLXhZrYMOWi8RICWajuAQHM5X3nQfitfg8MToMZCyZznPhbO0GPL4oHB5A8Rs5S8IS5bC1mAOx0Px0RMJAS+vu7+o2QHP/AFR8LiH5gAdyBsETQatnnvzrb4JBEdSjst1IuCC5EcUMpGiEkkigGKg5IlMgjJk9JIBkykopBp2nBQwVIFJvseMLsOx+BsmQjbQLlMHHmIA5lej8JgyMDR0W2E+uH8rk1PH+23DLWyWJmtppBZICGtG96nn5BUsXPo4D0VSOfH0p4jEhr7rTQX5cylNODqDoohucEHQNoFxG3X1WV7cD4GNHMAk6nfXVW1kX8XiWPtrIxeQNzOeGgHS3UdyddEODjj8M3u4wxxskvq83Qdcos77rKxgaG3fjvYHRo+p32VcYeSh0dRoee2ZLUXI0MFiTLNnlcXEAusncgHQfp5K/hMWYA4OfkzAkUPFl5ach+KzGSRwDUB7/AJBrhtpud1ReyWdxd7znamgdOVJVWtuo4bxvKWy6HK7XlZ5C+m4+a5qWLEGZ0UXFJQ25MgfLig4Q5HTRlxAILgGHYm9lUfK4UzW72HM7D1K3MPh3tMU7S1xYGva0HxkCi+MiuYLhr1U3GZHOhT2R417wxpJ8L/8AVz0XbPGoGjhqfksDibMZA7upeIgEWwh00riWnVrjbTsdfgF63wXFtfC1rXXlaA0nd8dUx/rWh6ODhyXAf2icIzP7wc9/VY2SFLarTdrXyQ4eN2IZ3zopMPLJmBDTG7PhpX5m1lumuPIZlS47heLYYMfio4Wd5T2GQcPIe4ACQA63YIPyRcP2Lww9kMmLFTBz5qIqGMBxsedMeNedIHaTtFicfgWwzNa44Z2ZkwGVzmZcha5gFXVGxW2yXhs5f6QdwHipZn9jjMZafH3GBLTGfGx+bLWXr5BU8Occ1zZWQQFzSyVpbDgyczTrRY3cmjY10Vni/YfPgY+IYJ7pIsgM0Tjb4njSSuoDr035qvweS4W1y0SmM+lLsXj8VSyMiiY/LL3rWOY17hFimh5aANQGPzAjkXhZfdYiteHN1ade5l3YdCKOugq0B5zYiQ+g/L9F0fY7sZFxAyRvkMTo2FsWWjmdmdIDIDsMsjaA3yO6J5Se4N6nbDMMu/8Ado3a/SLEDRw8WzutBE4NI+LERF2DyAPMbiGzjwSW0+88jZzt0FnCJMJjHQS1mYSCRqD0IRuNSubZDiPQkJST6fbncfhjFLJEd2Pcw/8AUkfRF4XHbs3T8z+yqoBceZJPxJK3oo2sFAV18z1U4Y7u1w+yG4p3PQ3OWqjOUaT0kSmDFRKRKgSkREJkrTIBJk6SASZOmQF0FEYhBHhCmNLXQ9mMJmkB5D8127hSxuzGCyxh1anVa2JOu/L9/iunGfHlct887S72lm4ubXdPiJlmyzLTS8IfEuIJ32H48z8/xVPKGa7qxjZ7BcdLv4jTf5LNfiDVfEetqdtpDyU4k6qZmLW00116n+irNepFylejMFGzr6psRO4mtvTokTqhOOqVUeaOvENevkVdixpb4rAOx+Rv6KgXHYFCll5+VH4pbPTpeDcdLCcp52Gk1WbdzXa5SaFggg8xzWrxHtE2RuR0F6eLMSzX+Wg4H1BXAM01taMUls97xDa+nNTqUrj3tCPhoc5zi11OIAGwOvUjbQcrPktiBlNpoAG1AaV9VRgdYBvbzRnYjUapySBXw+IxWC732WQtZKDnYW52aitWHb1GvkVzmCknjzloY4DV3jaANzepB5H5ELqpXkoMzrFOAPPUA7bKfCDTnOF4R7zncMrDrZ3ef0uytds0sUjJ4pHMewjVvMDUWNjufUEjmjOZ5qtK7dHjNaPQvaDiT8TP7S6JrXkAOMZNOI0Byu1aTW2vqsrjuK7wABpG16ak+XkPqrJlIrX4HUfJV3vvl9FHielXDwZfFVVsOfqUZz9tExSJT9KkRypqTlyG5yATioEpEqBKZESmSSQDJ1KkxSCKSekyASZJJAW2rR4XBne1vUrNYup7IYXM/NW2iMJ2XNl44122ChIZpyH5qliJF0MLBk2/Y/YXLcVNOcP3W4W/Hd2uDw1pSxUqol9pTSWFUz67rS1tjFjESAAtJ/Tl81QBRJ5LHppp+B/NVgVnWsgwKRchByRcltQhKZ5pQBQnyFTaZ3PQ5Taa0zlOzPacxnRMwohxWgaSDrd1qgDR7Vm0q1ZsUBqPRURIKq9kZknQ/BMDueol96oXeiqBslKIeFGwk56DI7RRLlHMkau6RQzKM+hQmlLZilQcU2qb4pBEkqDm+aKQouCACUyIQlSZB0tiPsxiS0ODWZnN7xsJlj9pcyswc2DNnILdQKsjYLJkGhC7/HdqYZn4ec4lsccLopXQNhlOKklhykMMxthBLTT8zQA6y0kAKcrSrz3OOoTh46hdNB2zkZkLcPEHMLyPeygvbM22sBGX/OJJB8RYzalHGdrnyRiP2eMAGcgjl37cQ01QFV7ST/4mba2boc5nHUfNNY6hdKe2ExvMxjg58L3hxccwga0d2TfuuLS5w5l7uqfGdrXyRTRHDxDvy1z3DN7wbE3MAbN/Yg+9u913afYcuUkimQa3GV6B2KgoNPXX4J0k8PrH8j5/13DNv36fRcLx55Ejh8fgkkr4b3UZT0xZJOSZ0R0A1JANfPRJJa5U4FKNORJ10O26qkpJKK0hZkWMNrXdJJSYUoINFCJSSU04iSouKSSSiY/ccioue0bX8UkkgG59okbz1SSQBarVWIXk6JJJgNx1UXOSSQFeRCY2kkkjRkchWkkgE0pF6SSARKZJJMiTUnSQEEkkkA4STpIARTJJJB//2Q=='
#context = '...'
messages= [ 
    dict(role="user", content=q, images=[context])
]
messages = [{'role': 'user', 'content': 'what is in this image', 'images':context}]
OLLAMA = Client(host=OLLAMA_HOST)

resp = ollama.chat( model= model, messages=messages)
resp

In [ ]:
#-------------------------------------------------------------------------------------------    
def tob64(imageFile):
    with open(imageFile, "rb") as f:
        b64Image = base64.b64encode(f.read()).decode("utf-8")
    return b64Image

file  = "/tmp/sat.jpeg"
context = tob64(file)

file  = "/tmp/people.jpg"
context1 = tob64(file)
q = "what is in this image"

context, context1

## Some RAG example

In [ ]:
prompt =  f'''
You are a knowledgeable AI assistant who answers a question from the context bellow. 
Given the following sections from the topic,
generate a final answer. Base your answer exclusively on the information.
If the context does not fully answer the query, dont make up an answer.

context: 
'''

def rag(q):
    pass

context = '''
1. Renewable energy is energy from natural sources that are replenished at a faster rate than they are consumed.
2. Solar energy is one of the most popular forms of renewable energy.
3. Wind energy is harnessed using turbines to generate electricity.
4. Hydropower relies on the flow of water to generate energy.
5. Renewable energy reduces greenhouse gas emissions and helps combat climate change.
'''

qs= ["What are the benefits of renewable energy?","Tell me more in bullet points?", 
     "did what you said make sense?", "can you summaerize what you said"]

msg = [{"role": "system", "content": prompt}]
for q in qs:
    msg.append( {"role": "user", "content": q } )
    resp=llm(messages=msg)
    msg.append(dict(role="assistant", content=resp))
    for mm in msg:
        pass; #print("\t",mm)
    print (f'\nq:{q}\n{resp}\n')


## To get the context going Example

In [ ]:
msg1=[
{"role": "system", "content": "You are ExploreAI, an outdoorsman's expert assistant."},
{"role": "user", "content": "What's the tallest mountain in the world?"},
{"role": "assistant", "content": "The tallest mountain in the world is Mount Everest, which reaches approximately 29,029 feet above sea level."},
{"role": "user", "content": "How does its height compare to the second tallest?"},
{"role": "assistant", "content": "Mount Everest is about 778 feet taller than K2, the second tallest mountain, which stands at approximately 28,251 feet above sea level."},
{"role": "user", "content": "Are they both part of the same mountain range?"},
{"role": "assistant", "content": "Yes, both Mount Everest and K2 are part of the Himalayan mountain range, which extends across five countries in Asia."},
{"role": "user", "content": "Which countries do they lie in?"},
{"role": "assistant", "content": "Mount Everest is located on the border between Nepal and China, in the Mahalangur Himal sub-range of the Himalayas. K2 is located on the China-Pakistan border, in the Baltistan region of the Karakoram range."},
{"role": "user", "content": "Is one harder to climb than the other?"}
]

msg=msg1[0:1]
for m in msg1[1::2]:
    msg.append(m)
    resp=rag(messages=msg)
    msg.append(dict(role="assistant", content=resp))
    for mm in msg:
        print("\t",mm)
    print ('\n')
